In [1]:
import numpy as np

# Introduction

We can use Newton's Method to find the roots of differentiable functions. The approach is to iterate from some starting guess of what the root is using the following formula:

\begin{equation*}
x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)}
\end{equation*}

where we let $x_0$ be our starting guess.

Theoretically, if $x_0$ is sufficiently close to the true root, say $r$ of the function under study, then $x_n \to r$ as $n \to \infty$. However, in practice establishing this initial guess is difficult, and numerical issues may cause this to fail. Here, I will explore this process and its behavior with an example function across a variety of parameters.

# Mathematical Formulation

In psuedo-code the algorithm we will use is this:

\begin{align*}
\text{Input:} &\quad x_0, f(x), f'(x), N, \epsilon_1, \epsilon_2 \\
\text{Output:} &\quad \alpha, flag \\
\text{1.}& \quad flag = True \\
\text{2.}& \quad \text{For} \; n = 1:N \\
&\quad \text{(a)} \quad f_x = f(x_0), \; df_x = f'(x_0) \\
&\quad \text{(b)} \quad \text{if} \; |df_x| \leq \epsilon_M \; \text{then} \; flag = False \; \text{then break} \\
&\quad \text{(c)} \quad \text{if} \; |f_x| \leq \epsilon_1 \; \text{then} \; \alpha = x_0, \; \text{then break} \\
&\quad \text{(d)} \quad \text{if} \; n \gt 0 \: \text{and} \; |x_1 - x_0| \leq \epsilon_2 \; \text{then} \; \alpha = x_0, \;\text{then break} \\
&\quad \text{(e)} \quad x_1 = x_0, \; x_0 = x_0 - \frac{f_x}{df_x}
\end{align*}

Where $\epsilon_M$ denotes the machine epsilon. Step (b) checks to see if the current derivative evaluation is close enough to zero that our step size would be zero. Step (c) checks to see if evaluating the function at the current guess is sufficiently close to zero, in which case we have found a root so we can stop. Step (d) checks to see if we are still making progress between iterations. If our last guess and our current guess are sufficiently close together, then we aren't making any better guess, so we can stop iterating.

# Numerical Experiments

In [115]:
def newton(x_0, f, df, N, eps_1, eps_2):
    flag = True
    x_1 = x_0
    for n in range(N):
        f_x = f(x_0)
        df_x = df(x_0)
        if abs(df_x) <= 10e-16:
            flag = False
            return flag
        elif abs(f_x) <= eps_1:
            return x_0
        elif (n > 0) and (abs(x_1 - x_0) <= eps_2):
            return x_0
        else:
            x_1 = x_0
            x_0 = x_0 - f_x / df_x

In [90]:
def f(x, B):
    return x + np.exp(-B * x**2) * np.cos(x)

def df(x, B):
    return -np.exp(-B * x**2) * np.sin(x) - 2 * B * x * np.exp(-B * x**2) * np.cos(x) + 1

In [98]:
Bs = [1, 5, 10, 25, 50]
x_0s = [-5, -1, 0, 1, 5]
results = {}

for B in Bs:
    x_results = {}
    for x_0 in x_0s:
        root = newton(x_0, lambda x: f(x, B), lambda x: df(x, B), 1000, 10e-15, 10e-15)
        x_results[x_0] = root
    results[B] = x_results

In [99]:
results

{1: {-5: -0.5884017765009963,
  -1: -0.5884017765009963,
  0: -0.5884017765009963,
  1: -0.5884017765009962,
  5: -0.5884017765009963},
 5: {-5: -0.4049115482093092,
  -1: -0.4049115482093092,
  0: -0.4049115482093092,
  1: -0.4049115482093092,
  5: -0.4049115482093092},
 10: {-5: None, -1: None, 0: None, 1: None, 5: None},
 25: {-5: None, -1: None, 0: None, 1: None, 5: None},
 50: {-5: None, -1: None, 0: None, 1: None, 5: None}}

| B | $x_0$ | Root |
| - | ----- | ---- |
| $1$ | $-5$ | $-0.5884017765009963$ |
| $1$ | $-1$ | $-0.5884017765009963$ |
| $1$ | $0$ | $-0.5884017765009963$ |
| $1$ | $1$ | $-0.5884017765009962$ |
| $1$ | $5$ | $-0.5884017765009963$ |
| $5$ | $-5$ | $-0.4049115482093092$ |
| $5$ | $-1$ | $-0.4049115482093092$ |
| $5$ | $0$ | $-0.4049115482093092$ |
| $5$ | $1$ | $-0.4049115482093092$ |
| $5$ | $5$ | $-0.4049115482093092$ |
| $10, 25, 50$ | $-5, -1, 0, 1, 5$ | N/A |

We can see from these results that for $B = 1, 5$ and any intial starting value, Newton's Method converges to the same root to a very high degree of precision. However, for values of $B \ge 10$ the algorithm does not converge for any starting value. This is because the algorithm gets "stuck" oscillating between $x_0 = -1$ and $x_0 = 0$. This is because $f'(x) \approx 1$ at those points. So, at each iteration if $x_n = 1$ then $x_{n+1} = 1 - 1 = 0$ and when $x_n = 0$ then $x_{n+1} = 0 - 1 = -1$. This doesn't happen for smaller values of $B$ because then the derivative at $x = -1$ is less close to $1$, so the next point is not exactly 0.

However, even for larger values of $B$, there are ranges of starting values that do result in convergence. For example, when $B=10$ and $x_0 = 0.8$ we get convergence:

In [122]:
newton(0.80, lambda x: f(x, 10), lambda x: df(x, 10), 1000, 10e-15, 10e-15)

-0.3264020100974987

However, once again, if we increase $B$ further, it fails to converge:

In [131]:
newton(0.80, lambda x: f(x, 25), lambda x: df(x, 25), 1000, 10e-15, 10e-15)

But, we can again find a starting value that allows for convergence:

In [132]:
newton(0.10, lambda x: f(x, 25), lambda x: df(x, 25), 1000, 10e-15, 10e-15)

-0.2374362439062778

In these cases with the right starting values, the algorithm doesn't get "trapped" in the loop between values of $1$ and $0$.

# Conclusion

We can see from these results that when the function and starting value are appropriate, Newton's method converges consistently and accurately to the roots of a function. However, we can also see that the method can be very sensitive to starting values.